# Setup

In [4]:
!pip install bitsandbytes accelerate # takes a while :-(  # restart kernel after installing this

In [34]:
from PIL import Image
import requests
from IPython.display import Image as IPImage
from IPython.display import Markdown

In [24]:
image_file = "https://static.wixstatic.com/media/ce79c2_1ba57b1f77604812b76db676c208eaaa~mv2.jpg"#/v1/fill/w_1500,h_650,al_c,q_90,usm_0.66_1.00_0.01,enc_auto/ce79c2_1ba57b1f77604812b76db676c208eaaa~mv2.jpg"
raw_image = Image.open(requests.get(image_file, stream=True).raw)

In [15]:
IPImage(url=image_file, width=1200)

# LLMs images (img -> text)

In [64]:
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
import json

In [3]:
model_id = "llava-hf/llava-1.5-7b-hf"

In [23]:
# converts the image to the proper input shape etc.
processor = AutoProcessor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
# setup 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_4bit=LlavaForConditionalGeneration.from_pretrained(model_id,device_map="auto",quantization_config=quantization_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
# Inference setup
prompt = "USER: <image>\nProvide a very detailed description of what can be seen and inferred from the image.\nASSISTANT:"
inputs = processor(prompt, raw_image, return_tensors='pt').to(0, torch.float16)
inputs

{'input_ids': tensor([[    1,  3148,  1001, 29901, 29871, 32000, 29871,    13,  1184, 29894,
           680,   263,  1407, 13173,  6139,   310,   825,   508,   367,  3595,
           322, 10115,  1127,   515,   278,  1967, 29889,    13, 22933,  9047,
         13566, 29901]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[[[ 1.1270e+00,  1.1270e+00,  9.0820e-01,  ...,  1.3027e+00,
            1.2588e+00,  1.2734e+00],
          [ 1.0840e+00,  1.0547e+00,  9.3750e-01,  ...,  1.4775e+00,
            1.3613e+00,  1.3174e+00],
          [ 9.3750e-01,  9.2285e-01,  9.0820e-01,  ...,  1.4922e+00,
            1.4189e+00,  1.4336e+00],
          ...,
          [ 1.1855e+00,  6.8945e-01, -2.8857e-01,  ...,  3.8281e-01,
            5.4346e-01,  6.1670e-01],
          [ 6.4551e-01, -3.9087e-01,  1.0553e-01,  ...,  8.6475e-01,
            8.7939e-01,  8

In [28]:
# run inference
output = model_4bit.generate(**inputs, max_new_tokens=750, do_sample=False)

In [47]:
output

tensor([[    1,  3148,  1001, 29901, 29871, 32000, 29871,    13,  1184, 29894,
           680,   263,  1407, 13173,  6139,   310,   825,   508,   367,  3595,
           322, 10115,  1127,   515,   278,  1967, 29889,    13, 22933,  9047,
         13566, 29901,   450,  1967,  5680,   263,  2318,   310,  2305, 29892,
          5517,  9121, 24127, 29892, 13407,   297,  4565,   310,   263,  5214,
         29889,  2688,   526, 27121,   297,  9090, 29879, 29892,   411,   777,
           310,   963,   591,  4362,   260,   583, 29889,   450,  2318, 11624,
           310,  1716,  1757,   322,  5866, 29892,   599, 13407,  4208,   297,
           263,  1196, 29889,    13,    13,  8439,   526,  3196,   282, 15048,
         18577,  7180,  2820,   278,  4038, 29892,  4417,   263,  6023,   310,
          7933,   708,   304,   278,  9088, 29889,  3834,   310,   278, 18577,
           526,  5982,  2978,   278,  5214, 29892,  1550,  4045,   526,  7180,
         17649,   304,   278,  2305, 29889,   450, 1

In [48]:
# added values:
output_tokens = output[0][inputs['input_ids'].shape[1]:]

In [92]:
# decode
Markdown(processor.decode(output_tokens, skip_special_tokens=True))

The image features a group of people, likely military personnel, standing in front of a building. They are dressed in uniforms, with some of them wearing ties. The group consists of both men and women, all standing together in a line.

There are several potted plants placed around the area, adding a touch of greenery to the scene. Some of the plants are located near the building, while others are placed closer to the people. The presence of the potted plants suggests that the location might be a military base or a similar setting.

In [54]:
def ask_a_question(userprompt, img):
  prompt = """USER: <image>\n{}\nASSISTANT:""".format(userprompt)
  inputs = processor(prompt, raw_image, return_tensors='pt').to(0, torch.float16)
  output = model_4bit.generate(**inputs, max_new_tokens=750, do_sample=False)
  output_tokens = output[0][inputs['input_ids'].shape[1]:]

  return processor.decode(output_tokens, skip_special_tokens=True)

In [65]:
my_custom_instruction = """How many females and males are in the picture? Answer in JSON"""

res = ask_a_question(my_custom_instruction, raw_image)

In [66]:
json.loads(res)

{'females': 4, 'males': 16}

In [73]:
my_custom_instruction = """Count the number persons in the picture. Answer in JSON"""

res = ask_a_question(my_custom_instruction, raw_image)

In [75]:
json.loads(res)

{'count': 21, 'men': 14, 'women': 7}

In [87]:
{"air-force" : ask_a_question("""How many light blue suits are in the picture? Answer in JSON""", raw_image),
 "army" : ask_a_question("""How many green suits are in the picture? Answer in JSON""", raw_image),
 "navy": ask_a_question("""How many people have a golden band on the sleaves? Answer in JSON""", raw_image)
 }



{'air-force': '10', 'army': '10', 'navy': '1'}